#### Check missing / duplicates

In [ ]:
import re
from pathlib import Path
import yaml
import pandas as pd

DETAILS_DIR = Path("/home/ubuntu/fast_llm_inference/RQ1_A30/details")

CONFIG_MODELS = """
# Qwen2.5
- Qwen/Qwen2.5-1.5B-Instruct
- Qwen/Qwen2.5-1.5B-Instruct-GPTQ-Int4
- Qwen/Qwen2.5-1.5B-Instruct-GPTQ-Int8
- Qwen/Qwen2.5-3B-Instruct
- Qwen/Qwen2.5-3B-Instruct-GPTQ-Int4
- Qwen/Qwen2.5-3B-Instruct-GPTQ-Int8
- Qwen/Qwen2.5-7B-Instruct
- Qwen/Qwen2.5-7B-Instruct-GPTQ-Int4
- Qwen/Qwen2.5-7B-Instruct-GPTQ-Int8
- Qwen/Qwen2.5-14B-Instruct-GPTQ-Int4
- Qwen/Qwen2.5-14B-Instruct-GPTQ-Int8
- Qwen/Qwen2.5-32B-Instruct-GPTQ-Int4

# Mistral
- mistralai/Mistral-7B-Instruct-v0.3
- marinarosell/Mistral-7B-Instruct-v0.3-GPTQ-8bit-gs128
- marinarosell/Mistral-7B-Instruct-v0.3-GPTQ-4bit-gs128
- dwetzel/Mistral-Small-24B-Instruct-2501-GPTQ-INT4

# Gemma-2
- google/gemma-2-2b-it
- marcinbrzezanski/gemma-2-2b-it-gptq
- RedHatAI/gemma-2-2b-it-quantized.w8a16
- google/gemma-2-9b-it
- shuyuej/gemma-2-9b-it-GPTQ
- RedHatAI/gemma-2-9b-it-quantized.w8a16
- shuyuej/gemma-2-27b-it-GPTQ

# LLaMA 3.x
- meta-llama/Llama-3.2-1B-Instruct
- clowman/Llama-3.2-1B-Instruct-GPTQ-Int4
- clowman/Llama-3.2-1B-Instruct-GPTQ-Int8
- meta-llama/Llama-3.2-3B-Instruct
- clowman/Llama-3.2-3B-Instruct-GPTQ-Int8
- clowman/Llama-3.2-3B-Instruct-GPTQ-Int4
- meta-llama/Llama-3.1-8B-Instruct
- clowman/Llama-3.1-8B-Instruct-GPTQ-Int4
- clowman/Llama-3.1-8B-Instruct-GPTQ-Int8
"""

# ---------------------------------------------------------------------------
expected_models = [m.split("/")[-1] for m in yaml.safe_load(CONFIG_MODELS)]
tasks = ["qa", "sql", "summarization"]

# 1️⃣  Scan all CSVs
existing_files = list(DETAILS_DIR.glob("*.csv"))

# helper → strip “vllm_” prefix and trailing “_<8-hex>.csv”
def normalize(fname: str) -> str:
    core = re.sub(r"^vllm_", "", fname)
    return re.sub(r"_[a-f0-9]{8}\.csv$", "", core)

# Build presence map and duplicate map
present = {m: {t: False for t in tasks} for m in expected_models}
dupes: dict[str, list[str]] = {}

for f in existing_files:
    norm = normalize(f.name)                       # e.g. Llama-..._qa_batch_bs16
    dupes.setdefault(norm, []).append(f.name)      # collect for duplicate check

    parts = norm.split("_")
    if len(parts) < 4:
        continue
    model_id = "_".join(parts[:-3])
    task      = parts[-3]
    if model_id in present and task in present[model_id]:
        present[model_id][task] = True

# 2️⃣  “Which runs are still missing?”
records = []
for model in expected_models:
    st = present[model]
    records.append(
        {"Model": model, "qa": st["qa"], "sql": st["sql"],
         "summarization": st["summarization"], "Complete": all(st.values())}
    )
missing_df = pd.DataFrame(records).sort_values("Model")

# 3️⃣  “Which runs are duplicated (same model+task, different hashes)?”
dup_records = [
    {"Normalized name": k, "Count": len(v), "Files": " | ".join(v)}
    for k, v in dupes.items() if len(v) > 1
]

if dup_records:
    dupes_df = pd.DataFrame(dup_records).sort_values("Count", ascending=False)
else:
    dupes_df = pd.DataFrame(columns=["Normalized name", "Count", "Files"])


# 4️⃣  Show / save results
print("\n=== Missing (incomplete) runs ===")
print(missing_df[~missing_df["Complete"]])

print("\n=== Duplicate CSVs (hash-agnostic) ===")
print(dupes_df if not dupes_df.empty else "No duplicates 🎉")

#### Merge run_report

In [20]:
#!/usr/bin/env python3
# merge_run_reports.py
# Usage: python merge_run_reports.py /home/ubuntu/fast_llm_inference/RQ1_merged_256/run_report

import sys
from pathlib import Path

import pandas as pd
import numpy as np

# ─────────────────────────────────────── config
RUN_DIR   = Path("/home/ubuntu/fast_llm_inference/RQ1_A30/run_report")
OUT_PATH  = RUN_DIR.parent / "merged_run_report.csv"

# All columns we want in the final file (order matters)
FINAL_COLS = [
    "model_name", "model_size_mb", "task", "scenario", "backend",
    "startup", "ttft_sec", "coldstart", "batch_size", "num_queries",
    "total_generation_time_s", "avg_gpu_mem_mb", "peak_gpu_mem_mb",
    "overhead_mb", "avg_gpu_util_pct", "peak_gpu_util_pct",
    "avg_cpu_util_pct", "peak_cpu_util_pct", "avg_ram_mb", "peak_ram_mb",
    "avg_power_w", "peak_power_w", "total_energy_wh",
    "avg_generation_time", "avg_tokens_generated", "avg_sentences_generated",
    "avg_ATL", "avg_GL", "avg_TPS", "avg_SPS",
    "avg_energy_per_token", "avg_energy_per_sentence"
]

# Quality‐metric columns we need for pick_quality()
QUALITY_COLS = ["avg_F1_score", "avg_AST_equal", "avg_ROUGE-1"]

def load_one(path: Path) -> pd.DataFrame:
    """Read one run_report CSV and normalize to FINAL_COLS + QUALITY_COLS schema."""
    df = pd.read_csv(path)

    # Ensure every expected column exists; missing ⇒ NaN
    for col in FINAL_COLS + QUALITY_COLS:
        if col not in df.columns:
            df[col] = np.nan

    # Keep only the final schema & order
    return df[FINAL_COLS + QUALITY_COLS]

# ─────────────────────────────────────── main
all_csvs = sorted(RUN_DIR.glob("*.csv"))
if not all_csvs:
    sys.exit(f"No CSVs found in {RUN_DIR}")

# Load & normalize each, then concatenate
merged = pd.concat([load_one(p) for p in all_csvs], ignore_index=True)

# ── write out ───────────────────────────────────────────────────────
merged.to_csv(OUT_PATH, index=False)
print(f"🚀  Merged {len(all_csvs)} files → {OUT_PATH}  ({len(merged)} rows)")

🚀  Merged 96 files → /home/ubuntu/fast_llm_inference/RQ1_A30/merged_run_report.csv  (96 rows)


#### Merge details

In [ ]:
#!/usr/bin/env python3
# merge_benchmark_details.py
# Usage:  python merge_benchmark_details.py /home/ubuntu/fast_llm_inference/RQ1_merged/details

import re
import sys
from pathlib import Path

import pandas as pd
import numpy as np

# ─────────────────────────────────────── config
DETAILS_DIR     = Path("/home/ubuntu/fast_llm_inference/RQ1_A30/details")
OUT_PATH        = DETAILS_DIR.parent / "merged_details.csv"

# All columns we want in the final file (order matters)
FINAL_COLS = [
    "backend", "model", "task",
    "generated_answer", "reference_answer",
    "generation_time", "tokens_generated", "sentences_generated",
    "ATL", "GL", "TPS", "SPS",
    "energy_per_token", "energy_per_sentence",
    "exact_match", "F1_score",
    "AST_equal", "Normalized_equal",
    "ROUGE-1", "ROUGE-2", "ROUGE-L"
]

# Regex to pull <backend>, <model>, <task> from the filename
PAT = re.compile(
    r"""^(?P<backend>[^_]+)_           # vllm, tgi …
        (?P<model>.+?)_                # greedy until _<task>_
        (?P<task>qa|sql|summarization) # capture task
        _[^/]*\.csv$                   # rest is scenario, hash …
    """, re.X | re.I)

def load_one(path: Path) -> pd.DataFrame:
    """Read a single details-CSV and normalise its columns."""
    m = PAT.match(path.name)
    if not m:
        raise ValueError(f"Cannot parse '{path.name}'")

    df = pd.read_csv(path)

    # Drop prompt – we don't need it in the merged file
    df = df.drop(columns=[c for c in df.columns if c.lower() == "prompt"], errors="ignore")

    # Inject backend / model / task from filename
    for k, v in m.groupdict().items():
        df[k] = v

    # Make sure every expected column exists; missing ⇒ NaN
    for col in FINAL_COLS:
        if col not in df.columns:
            df[col] = np.nan

    # Keep only the final schema & order
    return df[FINAL_COLS]

# ─────────────────────────────────────── main
all_csvs = sorted(DETAILS_DIR.glob("*.csv"))
if not all_csvs:
    sys.exit(f"No CSVs found in {DETAILS_DIR}")

merged = pd.concat([load_one(p) for p in all_csvs], ignore_index=True)
merged.to_csv(OUT_PATH, index=False)
print(f"🚀  Merged {len(all_csvs)} files → {OUT_PATH}  ({len(merged)} rows)")


🚀  Merged 96 files → /home/ubuntu/fast_llm_inference/RQ1_A30/merged_details.csv  (24576 rows)


#### Adding SQL execution accuracy 

In [ ]:


sql_task = SQLTask(
    db_root="/home/ubuntu/fast_llm_inference/benchmark/lookup/database",
    tables_path="/home/ubuntu/fast_llm_inference/benchmark/lookup/tables.json"
)
df_sql_id = pd.DataFrame(sql_task.generate_prompts(num_examples=256)[1:])
df_sql_id = df_sql_id.T            # flips rows↔columns
df_sql_id.columns = ["reference", "db_id"]

In [26]:
csv_path = "/home/ubuntu/fast_llm_inference/RQ1_A30/merged_details.csv"
merged = pd.read_csv(csv_path)

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# 1️⃣ Load full merged CSV
csv_path = "/home/ubuntu/fast_llm_inference/RQ1_A30/merged_details.csv"
merged_full = pd.read_csv(csv_path)

# 2️⃣ Build a mapping: reference → db_id
ref2db = dict(zip(df_sql_id["reference"], df_sql_id["db_id"]))

# 3️⃣ Add and fill db_id column
merged_full["db_id"] = pd.Series(dtype="object")
sql_mask = merged_full["task"].str.lower() == "sql"
merged_full.loc[sql_mask, "db_id"] = merged_full.loc[sql_mask, "reference_answer"].map(ref2db)

# 4️⃣ Filter SQL rows with valid db_id, excluding 'formula_1'
sql_eval = merged_full[
    (merged_full["task"].str.lower() == "sql") &
    (merged_full["db_id"].notnull()) &
    (merged_full["db_id"] != "formula_1")
].copy()

# 5️⃣ Evaluate SQL quality metrics
sql_eval.reset_index(drop=True, inplace=True)
sql_eval["Exec_accuracy"] = np.nan

for i, row in tqdm(sql_eval.iterrows(), total=len(sql_eval)):
    try:
        result = sql_task.quality_metrics(
            generated=row["generated_answer"],
            reference=row["reference_answer"],
            db_id=row["db_id"]
        )
        sql_eval.at[i, "Exec_accuracy"] = result.get("Exec_accuracy", np.nan)
    except Exception as e:
        print(f"❌ Row {i} failed: {e}")

# 6️⃣ Merge Exec_accuracy back into full DataFrame (safe for duplicate keys)
merged_full = pd.merge(
    merged_full,
    sql_eval[["reference_answer", "Exec_accuracy"]],
    on="reference_answer",
    how="left",
    suffixes=("", "_new")
)

# 7️⃣ Prioritize new values
merged_full["Exec_accuracy"] = merged_full["Exec_accuracy_new"].combine_first(merged_full["Exec_accuracy"])

# 8️⃣ Drop temp column
merged_full.drop(columns=["Exec_accuracy_new"], inplace=True)

# 9️⃣ (Optional) Save to disk
merged_full.to_csv("RQ1_A30_with_exec_accuracy.csv", index=False)y

  0%|          | 0/7936 [00:00<?, ?it/s]

100%|██████████| 7936/7936 [00:29<00:00, 269.60it/s]


KeyError: 'Exec_accuracy_new'

In [14]:
merged[(merged['model'] == "Mistral-Small-24B-Instruct-2501-GPTQ-INT4") & (merged['task'] == "sql")]["Exec_accuracy"].mean()

0.5282258064516129

In [18]:
merged.to_csv("/home/ubuntu/fast_llm_inference/RQ1_A30/merged_details_with_exec.csv", index=False)

#### Add GPU utilization

In [ ]:
import pandas as pd
import os

# ─── 1️⃣ Load df_report ──────────────────────────────────────────
df_report = pd.read_csv("/home/ubuntu/fast_llm_inference/RQ1_A30/merged_run_report.csv")
df_report["gpu_util_pct"] = None
df_report["gpu_util_pct_ci95"] = None

# ─── 2️⃣ File listing ────────────────────────────────────────────
readings_dir = "/home/ubuntu/fast_llm_inference/RQ1_A30/readings"
files = [f for f in os.listdir(readings_dir) if f.endswith(".csv") and f.startswith("ts_")]

matched, unmatched, skipped = 0, 0, 0

# ─── 3️⃣ Process each reading file ───────────────────────────────
for fname in files:
    path = os.path.join(readings_dir, fname)
    stem = fname.removesuffix(".csv")

    # Remove ts_ prefix
    if not stem.startswith("ts_"):
        skipped += 1
        continue
    stem = stem[3:]

    parts = stem.split("_")

    if len(parts) < 5:
        print(f"❌ Too few parts in filename: {fname}")
        skipped += 1
        continue

    # Extract metadata
    backend  = parts[0].strip().lower()
    model = parts[1].strip()
    task     = parts[2].strip().lower()
    scenario = parts[3].strip().lower()

    # Validate
    valid_tasks = {"sql", "qa", "summarization"}
    valid_scenarios = {"single", "batch", "server"}
    valid_backends = {"vllm", "tgi", "sglang", "lmdeploy"}

    if task not in valid_tasks or scenario not in valid_scenarios or backend not in valid_backends:
        print(f"⚠️ Unknown task or scenario in {fname}")
        skipped += 1
        continue

    # Read readings file
    try:
        df_reading = pd.read_csv(path)
    except Exception as e:
        print(f"❌ Could not read {fname}: {e}")
        skipped += 1
        continue

    if "gpu_util_pct" not in df_reading.columns:
        print(f"⚠️ Skipping {fname} — 'gpu_util_pct' missing")
        skipped += 1
        continue

    # Compute mean and CI95
    gpu_vals = df_reading["gpu_util_pct"].dropna()
    mean_val = gpu_vals.mean()
    std_val  = gpu_vals.std()
    n_val    = gpu_vals.count()
    ci95_val = 1.96 * std_val / (n_val ** 0.5) if n_val > 1 else 0.0

    # Find corresponding row
    mask = (
        (df_report["model_name"] == model)
        & (df_report["task"].str.lower() == task)
        & (df_report["scenario"].str.lower() == scenario)
        & (df_report["backend"].str.lower() == backend)
    )

    if mask.sum() == 1:
        df_report.loc[mask, "gpu_util_pct"] = mean_val
        df_report.loc[mask, "gpu_util_pct_ci95"] = ci95_val
        matched += 1
    elif mask.sum() == 0:
        print(f"❌ No match for model={model}, task={task}, scenario={scenario}, backend={backend}")
        unmatched += 1
    else:
        print(f"⚠️ Multiple matches for model={model}, task={task}, scenario={scenario}, backend={backend}")
        unmatched += 1

# ─── 4️⃣ Save updated report ─────────────────────────────────────
out_path = "/home/ubuntu/fast_llm_inference/RQ1_A30/merged_run_report_with_gpu_ci95.csv"
df_report.to_csv(out_path, index=False)

# ─── 5️⃣ Summary ─────────────────────────────────────────────────
print(f"✅ Done. Saved: {out_path}")
print(f"Matched:   {matched}")
print(f"Unmatched: {unmatched}")
print(f"Skipped:   {skipped}")

✅ Done. Saved: /home/ubuntu/fast_llm_inference/RQ1_A30/merged_run_report_info_with_gpu_ci95.csv
Matched:   96
Unmatched: 0
Skipped:   0


In [10]:
model_include = [
    "Mistral-7B-Instruct-v0.3", "Mistral-Small-24B-Instruct-2501-GPTQ-INT4",
    "gemma-2-9b-it", "gemma-2-27b-it-GPTQ",
    "Qwen2.5-7B-Instruct", "Qwen2.5-14B-Instruct-GPTQ-Int8", "Qwen2.5-32B-Instruct-GPTQ-Int4"
]

filtered_sizes = df_report[df_report['model_name'].isin(model_include)][['model_name', 'model_size_mb']]

filtered_sizes

,model_name,model_size_mb
33,Mistral-7B-Instruct-v0.3,13824.507812
34,Mistral-7B-Instruct-v0.3,13824.507812
35,Mistral-7B-Instruct-v0.3,13824.507812
36,Mistral-Small-24B-Instruct-2501-GPTQ-INT4,13492.045288
37,Mistral-Small-24B-Instruct-2501-GPTQ-INT4,13492.045288
38,Mistral-Small-24B-Instruct-2501-GPTQ-INT4,13492.045288
51,Qwen2.5-14B-Instruct-GPTQ-Int8,15875.072266
52,Qwen2.5-14B-Instruct-GPTQ-Int8,15875.072266
53,Qwen2.5-14B-Instruct-GPTQ-Int8,15875.072266
54,Qwen2.5-32B-Instruct-GPTQ-Int4,18447.634766


In [3]:
import pandas as pd
from pathlib import Path

# Path to the folder with CSVs
data_dir = Path("/home/ubuntu/fast_llm_inference/RQ3/details")
all_files = list(data_dir.glob("*.csv"))

# Filter for *_single_*.csv files
single_files = [f for f in all_files if "_single_" in f.stem]

# Load and annotate each CSV
df_list = []
for f in single_files:
    parts = f.stem.split("_")
    if len(parts) < 4:
        continue
    engine = parts[0]
    model = parts[1]
    task = parts[2]
    use_case = parts[3]

    df = pd.read_csv(f)
    df["engine"] = engine
    df["model"] = model
    df["task"] = task
    df["use_case"] = use_case
    df_list.append(df)

# Concatenate into one DataFrame
merged_single_df = pd.concat(df_list, ignore_index=True)

merged_single_df.to_csv("/home/ubuntu/fast_llm_inference/RQ3/merged_single_details.csv", index=False)

merged_single_df.columns

Index(['prompt', 'generated_answer', 'reference_answer', 'generation_time',
       'tokens_generated', 'sentences_generated', 'ATL', 'GL', 'TPS', 'SPS',
       'energy_per_token', 'energy_per_sentence', 'ROUGE-1', 'ROUGE-2',
       'ROUGE-L', 'engine', 'model', 'task', 'use_case', 'AST_equal',
       'exact_match', 'F1_score'],
      dtype='object')

In [8]:
# Re-import necessary modules due to code execution environment reset
import pandas as pd
from pathlib import Path

# Re-define path and find all CSVs again
data_dir = Path("/home/ubuntu/fast_llm_inference/RQ3/details")
all_files = list(data_dir.glob("*.csv"))

# Filter for *_batch_*.csv files
batch_files = [f for f in all_files if "_batch_" in f.stem]

# Load and annotate each CSV
batch_df_list = []
for f in batch_files:
    parts = f.stem.split("_")
    if len(parts) < 4:
        continue
    engine = parts[0]
    model = parts[1]
    task = parts[2]
    use_case = parts[3]
    batch_size = parts[4]

    df = pd.read_csv(f)
    df["engine"] = engine
    df["model"] = model
    df["task"] = task
    df["use_case"] = use_case
    df["batch_size"] = batch_size
    batch_df_list.append(df)

# Concatenate into one DataFrame
merged_batch_df = pd.concat(batch_df_list, ignore_index=True)

merged_batch_df.to_csv("/home/ubuntu/fast_llm_inference/RQ3/merged_batch_details.csv", index=False)

merged_batch_df


,prompt,generated_answer,reference_answer,generation_time,tokens_generated,sentences_generated,ATL,GL,TPS,SPS,...,engine,model,task,use_case,batch_size,ROUGE-1,ROUGE-2,ROUGE-L,exact_match,F1_score
0,### SYSTEM\nYou are a SQL query generation ass...,"SELECT Band.Firstname, Band.Lastname FROM Perf...","SELECT T2.firstname , T2.lastname FROM Perfor...",0.185022,13,2,0.014232,0.185022,70.26,10.81,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs32,NaN,NaN,NaN,NaN,NaN
1,### SYSTEM\nYou are a SQL query generation ass...,SELECT policy_type_code FROM Available_Policies,SELECT DISTINCT t3.policy_type_code FROM custo...,0.185022,12,1,0.015419,0.185022,64.86,5.40,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs32,NaN,NaN,NaN,NaN,NaN
2,### SYSTEM\nYou are a SQL query generation ass...,"SELECT station.id, station.name","SELECT DISTINCT T1.id , T1.name FROM station ...",0.185022,9,2,0.020558,0.185022,48.64,10.81,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs32,NaN,NaN,NaN,NaN,NaN
3,### SYSTEM\nYou are a SQL query generation ass...,SELECT customer_name FROM Customers WHERE cust...,SELECT customer_name FROM customers WHERE cust...,0.185022,18,1,0.010279,0.185022,97.29,5.40,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs32,NaN,NaN,NaN,NaN,NaN
4,### SYSTEM\nYou are a SQL query generation ass...,SELECT technician.Name,SELECT T3.Name FROM repair_assignment AS T1 JO...,0.185022,6,1,0.030837,0.185022,32.43,5.40,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs32,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35835,### SYSTEM\nYou are a SQL query generation ass...,SELECT Advisor FROM Student GROUP BY Advisor H...,SELECT Advisor FROM STUDENT GROUP BY Advisor H...,0.138727,23,1,0.006032,0.138727,165.79,7.21,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs64,NaN,NaN,NaN,NaN,NaN
35836,### SYSTEM\nYou are a SQL query generation ass...,"SELECT state, SUM(acc_bal) as total_balance","SELECT sum(acc_bal) , state FROM customer WHE...",0.138727,15,1,0.009248,0.138727,108.13,7.21,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs64,NaN,NaN,NaN,NaN,NaN
35837,### SYSTEM\nYou are a SQL query generation ass...,"SELECT artist_name, most_popular_in",SELECT artist_name FROM song WHERE resolution ...,0.138727,12,1,0.011561,0.138727,86.50,7.21,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs64,NaN,NaN,NaN,NaN,NaN
35838,### SYSTEM\nYou are a SQL query generation ass...,"SELECT Employees.Employee_ID, COUNT(All_Docume...","SELECT Destroyed_by_Employee_ID , count(*) FR...",0.138727,33,2,0.004204,0.138727,237.88,14.42,...,vllm,Mistral-7B-Instruct-v0.3,sql,batch,bs64,NaN,NaN,NaN,NaN,NaN


In [9]:
merged_batch_df['model'].unique()

array(['Mistral-7B-Instruct-v0.3', 'Qwen2.5-3B-Instruct',
       'Llama-3.1-8B-Instruct'], dtype=object)

In [9]:
merged_server_df.head(1)

,user_id,scheduled_ts,submit_time,send_time,start_time,queue_time,wait_time,e2e_latency,generated_answer,reference_answer,...,ROUGE-2,ROUGE-L,engine,model,task,use_case,concurrent_users,exact_match,F1_score,AST_equal
0,5,30.356503,1.751223e+09,1.751223e+09,1.751223e+09,19.540525,19.540696,29.187121,Liverpool advanced to the FA Cup semi-finals w...,Coutinho hit the only goal of the game as Live...,...,0.109375,0.184615,tgi,Llama-3.1-8B-Instruct,summarization,server,16,NaN,NaN,NaN
